In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import random

In [ ]:
# Function: Get website name from a URL
def get_website_name(url):
    parsed_url = urlparse(url)
    # Get the netloc attribute, containing the domain name
    website_name = parsed_url.netloc
    # Remove 'www.' prefix if present
    if website_name.startswith('www.'):
        website_name = website_name[4:]
    return website_name

# TESTING
URL = "https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier"
print(get_website_name(URL))

eurogamer.net


In [ ]:
# Function: Counting the number of name in link_set
def count_links_containing_website_name(link_set, name):
    return sum(name in string for string in link_set)

# TESTING
a = {'https://www.eurogamer.net/stardew-valley-creator-says-16-update-content-is-done-definitely-out-this-year', 'https://www.reddit.com/r/StardewValley/comments/17y5i0p/bathhousespa_decoration/', 'https://www.eurogamer.net/stardew-valley-creator-teases-an-important-addition-in-16-hats-on-cats-and-dogs', 'https://www.eurogamer.net/stardew-valley-patch-16-is-on-the-way?view=comments', 'https://www.eurogamer.net/stardew-valley-creator-confirms-hes-made-a-ton-of-progress-on-update-16?view=comments', 'https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier?view=comments', 'https://www.eurogamer.net/this-speedrunner-completed-stardew-valley-in-just-17-minutes', 'https://www.eurogamer.net/stardew-valley-creator-shares-a-selection-of-adorable-hand-drawn-concept-notes-and-sketches', 'https://www.eurogamer.net/stardew-valley-patch-16-is-on-the-way', 'https://www.eurogamer.net/stardew-valley-creator-doesnt-know-if-there-will-be-another-update-focused-on-next-game', 'https://www.eurogamer.net/new-haunted-chocolatier-screenshot-suggests-stardew-valley-grandpa-may-return', 'https://www.eurogamer.net/stardew-valley-creator-pauses-haunted-chocolatier-development', 'https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier', 'https://www.eurogamer.net/this-stardew-valley-player-has-figured-out-how-to-decorate-the-default-in-game-spa', 'https://www.eurogamer.net/stardew-valley-creator-teases-version-16-content', 'https://en.wikipedia.org/wiki/Stardew_Valley', 'https://www.eurogamer.net/stardew-valley-16-includes-new-late-game-content-and-another-major-festival', 'https://assetsio.reedpopcdn.com/stardew-valleys-getting-even-more-free-content-1582805780346.jpg?width=1920', 'http://gamasutra.com/view/news/267563/The_4_years_of_selfimposed_crunch_that_went_into_Stardew_Valley.php', 'http://www.eurogamer.net/articles/2016-11-29-stardew-valley-is-coming-to-xbox-one-in-two-weeks', 'https://www.eurogamer.net/stardew-valley-creator-confirms-hes-made-a-ton-of-progress-on-update-16'}
count_links_containing_website_name(a, "eurogamer.net")

17

In [ ]:
# Adapted code from https://github.com/kjmazidi/NLP/blob/master/Xtra_Python_Material/Web_Scraping/3%20-%20Web%20crawler%20-%20almost.ipynb
# Instead of writing to a file, append to a list

# Function: Scrape all URLs from an initial URL
def get_urls(url, exclude_links = ['google', 'wiki', 'mod', 'png']):
  # Initialize an empty list to store extracted URLs
  url_list = []

  try:
    # Make a GET request to the provided URL
    r = requests.get(url)

    # Check if the request was successful (status code 200)
    if r.status_code == 200:
      # Parse the HTML of the link
      soup = BeautifulSoup(r.text, 'html.parser')

      # Find all <a> tags containing links
      for link in soup.find_all('a'):
        # Get the value of the 'href' attribute of the <a> tag
        link_str = str(link.get('href'))

        # Check if the link contains certain keywords
        if ('Stardew' in link_str or 'stardew' in link_str) and ('Valley' in link_str or 'valley' in link_str):
          # Remove unnecessary parameters from the URL (if present)
          if link_str.startswith('/url?q='):
            link_str = link_str[7:]

          # Remove additional parameters after '&' (if present)
          if '&' in link_str:
            i = link_str.find('&')
            link_str = link_str[:i]

          # Exclude URLs containing 'google' or 'wiki' or 'mod'
          if link_str.startswith('http') and not any(exclude_link in link_str for exclude_link in exclude_links):
            url_list.append(link_str)
  except:
    print("Could not extract links from", url)

  # Return the list of extracted URLs
  return url_list



In [ ]:
#Function: Return n random elements from a list
def get_n_random_elements(list, NUMBER_OF_ELEMENTS = 2):
    # If list less than NUMBER_OF_ELEMENTS elements, return the list back
    if len(list) < NUMBER_OF_ELEMENTS:
        return list

    # Select NUMBER_OF_ELEMENTS random elements from the list
    else:
      random_elements = random.sample(list, NUMBER_OF_ELEMENTS)
      return random_elements

# TESTING
test = [1, 2, 3, 4, 5]
test_elements = get_n_random_elements(test, 2)
print(test_elements)

[4, 5]


In [ ]:
def crawler(initial_urls, MAX_SIZE = 25, URL_PER_LINK = 3):
  # MAX_SIZE is the number of relevent sites
  # URL_PER_LINK is the number of URLs crawled for each unique site visited
  url_queue = []
  url_queue.extend(starter_urls)

  # This visited links set makes sure that there are no repeats in the sites
  visited_links = set()
  visited_links.update(initial_urls)

  # This visited website set keeps the website name
  # visited_websites_name = set()
  # visited_websites_name.add(get_website_name(initial_url))

  # Manually removed links which resulted in poor data (from experience)
  exclude_links = ['google', 'wiki', 'linkedin', 'facebook', 'mod', 'png', 'jpg', 'jpeg', 'instagram']
  while True:
    if len(visited_links) >= MAX_SIZE:
      return visited_links

    # Case where the queue is empty before exiting the function
    if not url_queue:
      # Add the initial URL back
      url_queue.extend(starter_urls)

    # Pop the top URL of the queue
    current_url = url_queue.pop(0)

    visited_links.add(current_url)
    scraped_urls = get_urls(current_url, exclude_links)

    # Filter out already visited URLs
    # scraped_urls = [url for url in scraped_urls if url not in visited_links]

    n_urls = get_n_random_elements(scraped_urls, URL_PER_LINK)

    for url in n_urls:
      try:
        r = requests.get(url)
        if r.status_code == 200:
          website_name = get_website_name(url)
          exclude_links.append(website_name)
          # Append remaining URLs to the queue
          url_queue.append(url)
          print("Added to the list:", url)
      except:
        print("Could not add to the list", url)

In [ ]:
starter_urls = ["https://en.wikipedia.org/wiki/Stardew_Valley", "https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier"]
#get_urls(starter_url)

urls = crawler(starter_urls)

Added to the list: https://web.archive.org/web/20170705220431/http://www.siliconera.com/2017/02/28/chucklefish-bringing-wargroove-stardew-valley-pocket-rumble-switch/
Added to the list: https://web.archive.org/web/20170718172936/http://www.gamasutra.com/view/news/267563/The_4_years_of_selfimposed_crunch_that_went_into_Stardew_Valley.php
Added to the list: https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier?view=comments
Added to the list: https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier?view=comments
Added to the list: https://www.eurogamer.net/stardew-valleys-long-awaited-16-update-gets-march-release-date-on-pc
Added to the list: http://www.siliconera.com/2017/01/19/stardew-valley-vita-port-investigation/
Added to the list: http://www.gameinformer.com/b/features/archive/2016/04/28/understanding-autism-through-stardew-valley.aspx
Could not add to the list https://gamerant.com/stardew-valley-greenhouse-guide-unlock-layout-crops/
Added

In [ ]:
print("Number of links:", len(urls))
print("Links:")
for url in urls:
  print(url)

Number of links: 25
Links:
https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier
https://blog.us.playstation.com/2016/12/08/country-life-rpg-stardew-valley-coming-to-ps4-on-december-13/#comment-1193243
http://www.gameinformer.com/b/features/archive/2016/04/28/understanding-autism-through-stardew-valley.aspx
https://web.archive.org/web/20170705220431/http://www.siliconera.com/2017/02/28/chucklefish-bringing-wargroove-stardew-valley-pocket-rumble-switch/
https://www.destructoid.com/stardew-valley-is-headed-to-ios-this-month-and-then-android-526438.phtml
https://www.gamesradar.com/stardew-valley-update-16-drops-next-month-concernedape-confirms-now-back-to-the-grind/
https://www.polygon.com/2018/5/14/17352396/stardew-valley-ps-vita-release-date
https://www.pushsquare.com/games/ps4/stardew_valley
https://en.wikipedia.org/wiki/Stardew_Valley
https://www.pcgamer.com/stardew-valley-multiplayer-out-now-on-pc/
https://www.vg247.com/2017/08/01/stardew-valley-multiplayer-det

In [ ]:
import re
def scrape_and_write_to_files(urls):
  file_id = 1
  for url in urls:
    try:
      # Fetch the HTML content of the URL
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for HTTP errors

    except Exception as e:
      print(e)

    # Parse the HTML of the link
    soup = BeautifulSoup(response.text, 'html.parser')

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
      for script in soup(["script", "style"]):
        script.extract()
      # Extract text from the HTML
      # text = soup.get_text()

      # Extract text from all paragraphs on the webpage
      # Find all <p> elements without any classes specified
      paragraphs = soup.select('p:not([class])')

      # Initialize a variable to store the extracted text
      text = ''

      for paragraph in paragraphs:
        text += paragraph.get_text() + ' '

      # Creating the filename from the URL
      # Returns the website name except for the top level domain


      # Only if text exists, (if blank do nothing)
      if text:
        split_website = get_website_name(url).split('.')
        file_name = split_website[:-1]
        file_name = '-'.join(file_name)
        file_name = file_name.replace('.', '-')

        # Add the ID
        file_name += ('-raw-' + str(file_id))
        file_id   += 1

        # Add the .txt extention
        file_name += '.txt'

        # Add path
        file_path = "raw/" + file_name
        print(file_name)

        with open(file_path, 'w', encoding='utf-8') as file:
          file.write(text)

        print("Text scraped from %s and written to %s" % (url, file_name))

    else:
      print("Website fetch failed.")

In [ ]:
# Create "raw" folder on current directory:
import os

folder_name = "raw"

# Check if the folder exists
if not os.path.exists(folder_name):
  # If it doesn't exist, create it
  os.makedirs(folder_name)
  print("Folder %s created." %folder_name)
else:
  print("Folder %s exists." %folder_name)

scrape_and_write_to_files(urls)

Folder raw created.
eurogamer-raw-1.txt
Text scraped from https://www.eurogamer.net/stardew-valley-creator-reveals-haunted-chocolatier and written to eurogamer-raw-1.txt
blog-us-playstation-raw-2.txt
Text scraped from https://blog.us.playstation.com/2016/12/08/country-life-rpg-stardew-valley-coming-to-ps4-on-december-13/#comment-1193243 and written to blog-us-playstation-raw-2.txt
gameinformer-raw-3.txt
Text scraped from http://www.gameinformer.com/b/features/archive/2016/04/28/understanding-autism-through-stardew-valley.aspx and written to gameinformer-raw-3.txt
web-archive-raw-4.txt
Text scraped from https://web.archive.org/web/20170705220431/http://www.siliconera.com/2017/02/28/chucklefish-bringing-wargroove-stardew-valley-pocket-rumble-switch/ and written to web-archive-raw-4.txt
destructoid-raw-5.txt
Text scraped from https://www.destructoid.com/stardew-valley-is-headed-to-ios-this-month-and-then-android-526438.phtml and written to destructoid-raw-5.txt
gamesradar-raw-6.txt
Text s

In [ ]:
import string

def ends_with_punctuation(text):
  # Get the last character of the text
  last_character = text[-1]

  # Check if  last character is a punctuation
  if last_character in string.punctuation:
    return True
  else:
    return False

# TEST
sentence1 = "^ This does end with a period."
sentence2 = "Nope, this one doesn't"
sentence3 = "Yup, this one doesn't, but ends with a question mark?"

print(ends_with_punctuation(sentence1))
print(ends_with_punctuation(sentence2))
print(ends_with_punctuation(sentence3))

True
False
True


In [ ]:
def starts_with_char(text, char):
  text = text.strip()
  if text.startswith(char):
    return True
  else:
    return False

# TEST
sentence1 = "^ This does start with a ^."
sentence2 = "Nope, this one doesn't"

print(starts_with_char(sentence1, '^'))
print(starts_with_char(sentence2, '^'))

True
False


In [ ]:
def count_words(sentence):
  # Split sentence to word
  words = sentence.split()

  # Count number of words
  num_words = len(words)

  return num_words

In [ ]:
def clean_up_file(text):

  # Remove ['num']
  filtered_sentence = re.sub(r'\[\d+\]', '', text)

  # Remove URLs
  filtered_sentence = re.sub(r'https?:\/\/\S*', '', filtered_sentence)

  # Remove whitespace
  filtered_sentence = re.sub(r'^\s*$', '', filtered_sentence)

  # Remove ©
  filtered_sentence = re.sub(r'©', '', filtered_sentence)

  # Sentence boundaries
  # filtered_sentence = re.sub(r'([.?!0-9])([A-Z])', r'\1 \2', filtered_sentence)

  # Remove leading whitespaces
  text_chunks = [chunk.strip() for chunk in filtered_sentence.splitlines()]

  cleaned_up_text = ""

  for i, chunk in enumerate(text_chunks):
    # Makes sure that doesn't start with '^' and chunk is larger than 5 words
    if not starts_with_char(chunk, '^') and count_words(chunk) > 5:
      # Adds a '.' if the sentence doesn't end in a punctuation
      if not ends_with_punctuation(chunk):
        chunk += '.'
      # print(i+1, chunk)
      cleaned_up_text += chunk + ' '

  return cleaned_up_text

In [ ]:
# Testing clean_up_file(text)

test_file_folder = "raw/en-wikipedia-raw-9.txt"
with open(test_file_folder, 'r') as file:
  file_contents = file.read()
  # print(file_contents)
  cleaned_up = clean_up_file(file_contents)

print(cleaned_up)

Stardew Valley is a 2016 farm life sim game developed by Eric "ConcernedApe" Barone. Players take the role of a character who inherits their deceased grandfather's dilapidated farm in a place known as Stardew Valley. The game was released for Windows in February 2016 before being ported to other platforms. Stardew Valley is open-ended, allowing players to grow crops, raise livestock, fish, cook, mine, forage, and socialize with the townspeople, including the ability to marry and have children. It allows up to four players to play online together. Barone developed Stardew Valley by himself over four and a half years. He was heavily inspired by the Story of Seasons series, with additions to address some of the shortcomings of those games. He used it as an exercise to improve his own programming and game design skills. British studio Chucklefish approached Barone midway through development with the offer to publish the game, allowing him to focus more on completing it. Stardew Valley has 

In [ ]:
# Writing x cleaned files
# -----------------------------

# Create new folder
proc_folder_name = 'process/'

# Check if the folder exists
if not os.path.exists(proc_folder_name):
  # If it doesn't exist, create it
  os.makedirs(proc_folder_name)
  print("Folder %s created." %proc_folder_name)
else:
  print("Folder %s exists." %proc_folder_name)

# Get the folder that holds the raw data
raw_dir = "raw/"

# Get all file names from raw_dir
file_names = os.listdir(raw_dir)

for name in file_names:
  curr_file = raw_dir + name
  # print(curr_file)

  # Opening each file in "raw" and reading its content
  with open(curr_file, 'r') as file:
    # print(curr_file)
    file_contents = file.read()
    clean_contents = clean_up_file(file_contents)
    # print(clean_contents)
    # print("-------------------------------------------------")

  # Writing the cleaned file to a different folder


  #Remove raw from name
  name = name.replace("-raw", "-clean")


  processed_file = proc_folder_name + name
  # print(processed_file)
  with open(processed_file, 'w', encoding='utf-8') as file:
    file.write(clean_contents)
    print("Text cleaned and written to %s" % (processed_file))

Folder process/ created.
Text cleaned and written to process/pcgamesn-clean-18.txt
Text cleaned and written to process/destructoid-clean-5.txt
Text cleaned and written to process/nintendolife-clean-16.txt
Text cleaned and written to process/eurogamer-clean-1.txt
Text cleaned and written to process/gamesradar-clean-6.txt
Text cleaned and written to process/polygon-clean-7.txt
Text cleaned and written to process/pushsquare-clean-8.txt
Text cleaned and written to process/blog-us-playstation-clean-2.txt
Text cleaned and written to process/gamespot-clean-12.txt
Text cleaned and written to process/eurogamer-clean-13.txt
Text cleaned and written to process/blog-playstation-clean-19.txt
Text cleaned and written to process/web-archive-clean-4.txt
Text cleaned and written to process/forbes-clean-20.txt
Text cleaned and written to process/eurogamer-clean-22.txt
Text cleaned and written to process/gameinformer-clean-3.txt
Text cleaned and written to process/purexbox-clean-15.txt
Text cleaned and w